## Find the optimal starting word for Wordle.

In [2]:
# read in external file
f = open(r"C:\Users\ethan\OneDrive\UT Courses\BAX 305\valid_solutions.csv", 'r', encoding = 'utf-8')
lines = f.readlines()

# clean the lines up to have only the word on each lines
lines = [line.strip('\n') for line in lines]
for line in lines:
    if len(line) != 5:
        lines.remove(line)

# create the alphabet
letters = []
for line in lines:
    if line[2] not in letters:
        letters.append(line[2])

# double check that the alphabet is correct
len(letters) == 26

True

In [3]:
D = {}
frequencies = []


for index in range(0,5): 
    # create list 
    for letter in letters:
        letter_words = []
        for line in lines:
            if len(line) == 5:
                if line[index] == letter:
                    letter_words.append(line)
    
        D[letter] = letter_words

    total_count = len([line for line in lines if len(line) == 5])
    D_pct = {}

    for letter,words in D.items():
        letter_count = len(words)
        letter_pct = letter_count / total_count * 100
        D_pct[letter] = letter_pct

    frequencies.append(dict(sorted(D_pct.items(), key = lambda d: d[1], reverse = True)))

In [4]:
# which words are best at resulting in green squares?
green_words = {}

for line in lines:
    total_freq = 0
    
    for index in range(len(line)): 
        char = line[index]
        freq = frequencies[index][char]
        total_freq += freq
    
    green_words[line] = round(total_freq,1)
        
sorted(green_words.items(), key = lambda d: d[1], reverse = True)[0:5]

[('slate', 62.1),
 ('sauce', 61.0),
 ('slice', 60.9),
 ('shale', 60.6),
 ('saute', 60.4)]

In [5]:
# create dictionaries
freq_one = {}
freq_two = {}
freq_three = {}

# populate dictionaries
for letter in letters:
    one_count = 0
    two_count = 0
    three_count = 0
    
    for line in lines:
        appearances = 0
        
        # count number of lines 
        if letter in line:
            appearances = line.count(letter)
        if appearances == 3:
            one_count += 1
            two_count += 1
            three_count += 1
        elif appearances == 2:
            one_count += 1
            two_count += 1
        elif appearances == 1:
            one_count += 1
            
        freq1 = one_count/len(lines)
        freq2 = two_count/len(lines)
        freq3 = three_count/len(lines)
        
    freq_one[letter] = round(freq1*100,1)
    freq_two[letter] = round(freq2*100,1)
    freq_three[letter] = round(freq3*100,1)

# return the letters that appear in a word at least once most commonly
sorted(freq_one.items(), key = lambda d: d[1], reverse = True)[:5]

[('e', 45.6), ('a', 39.3), ('r', 36.2), ('o', 29.1), ('t', 28.8)]

In [6]:
yellow_words = {}

for line in lines:
    total_freq = 0
    
    # 
    for letter in letters:
        if line.count(letter) == 3:
            total_freq = total_freq + freq_three[letter] + freq_two[letter] + freq_one[letter]
        elif line.count(letter) == 2:
            total_freq = total_freq + freq_two[letter] + freq_one[letter]
        elif line.count(letter) == 1:
            total_freq = total_freq + freq_one[letter]
        
    avg_freq = total_freq/5
    
    yellow_words[line] = round(avg_freq,1)
    
sorted(yellow_words.items(), key = lambda d: d[1], reverse = True)[:5]

[('alert', 35.6),
 ('alter', 35.6),
 ('irate', 35.6),
 ('later', 35.6),
 ('arose', 35.4)]

In [7]:
word_scores = dict(green_words)

for word,score in dict(yellow_words).items():
    word_scores[word] += score

sorted(word_scores.items(), key = lambda d: d[1], reverse = True)[:10]

[('slate', 95.80000000000001),
 ('share', 93.0),
 ('stare', 92.6),
 ('crane', 92.4),
 ('saute', 92.4),
 ('saner', 92.1),
 ('snare', 92.0),
 ('shale', 91.8),
 ('crate', 91.6),
 ('stale', 91.4)]

In [10]:
# how good was my previous starting word, stern?
for index in range(len(sorted(word_scores.items(), key = lambda d: d[1], reverse = True))):
    if sorted(word_scores.items(), key = lambda d: d[1], reverse = True)[index][0] == 'adieu':
        print(index)
        print(sorted(word_scores.items(), key = lambda d: d[1], reverse = True)[index])

In [56]:
# build a function to return all the possible words left

def possibilities(words,feedback):
    """
    This function expects two lists as input. The first list should be all the words guessed as strings (up to five).
    The second list should be the color received from each of those letters. Colors should be denoted with one letter:
    G for green, Y for yellow, N for nothing. 
    
    For example:
    If my first guess was SLATE, the S was green, the T was yellow, and no other colors showed up, 
    I would call the function this way:
    possibilities(['SLATE'],['GNNYN'])
    
    If I then were to guess STINK, the S, T, I, and K were green, and the N showed nothing,
    I would call the function this way:
    possibilities(['SLATE','STINK'],['GNNYN','GGGNG'])
    """
    remaining = lines
    
    if len(words) != len(feedback):
        return "Length of the words and feedback lists must be equal"
    
    for index in range(len(feedback)):
        for letter in feedback[index]:
            if letter not in "GYN":
                return "Please only use the letters G, Y, and N as feedback"
    
    for line in lines:
        for windex in range(len(words)):
            for lindex in range(5):
                if feedback[windex][lindex] == "G":
                    if line[lindex] != words[windex][lindex]:
                        if line in remaining:
                            remaining.remove(line)
                elif feedback[windex][lindex] == "Y":
                    if line[lindex] == words[windex][lindex] or words[windex][lindex] not in line:
                        if line in remaining:
                            remaining.remove(line)
                elif feedback[windex][lindex] == "N":
                    if words[windex][lindex] in line:
                        if line in remaining:
                            remaining.remove(line)
    
    return remaining

In [57]:
possibilities(['SLATE'],['NNNNN'])

['aback',
 'abase',
 'abate',
 'abbey',
 'abbot',
 'abhor',
 'abide',
 'abled',
 'abode',
 'abort',
 'about',
 'above',
 'abuse',
 'abyss',
 'acorn',
 'acrid',
 'actor',
 'acute',
 'adage',
 'adapt',
 'adept',
 'admin',
 'admit',
 'adobe',
 'adopt',
 'adore',
 'adorn',
 'adult',
 'affix',
 'afire',
 'afoot',
 'afoul',
 'after',
 'again',
 'agape',
 'agate',
 'agent',
 'agile',
 'aging',
 'aglow',
 'agony',
 'agora',
 'agree',
 'ahead',
 'aider',
 'aisle',
 'alarm',
 'album',
 'alert',
 'algae',
 'alibi',
 'alien',
 'align',
 'alike',
 'alive',
 'allay',
 'alley',
 'allot',
 'allow',
 'alloy',
 'aloft',
 'alone',
 'along',
 'aloof',
 'aloud',
 'alpha',
 'altar',
 'alter',
 'amass',
 'amaze',
 'amber',
 'amble',
 'amend',
 'amiss',
 'amity',
 'among',
 'ample',
 'amply',
 'amuse',
 'angel',
 'anger',
 'angle',
 'angry',
 'angst',
 'anime',
 'ankle',
 'annex',
 'annoy',
 'annul',
 'anode',
 'antic',
 'anvil',
 'aorta',
 'apart',
 'aphid',
 'aping',
 'apnea',
 'apple',
 'apply',
 'apron',


In [52]:
len(lines)

2315